In [1]:
import sys
import numpy as np
from tqdm import tqdm
import csv
import random
import matplotlib.pyplot as plt
from model import *
from view import View
from data_cleaning import all_groups, data_array

<IPython.core.display.Javascript object>

DATA CLEAN fully OBJECT Columns: ['EPOCH', 'INCLINATION', 'RA_OF_ASC_NODE', 'ARG_OF_PERICENTER', 'MEAN_ANOMALY', 'NORAD_CAT_ID', 'SEMIMAJOR_AXIS', 'OBJECT_TYPE', 'RCS_SIZE', 'LAUNCH_DATE', 'tuples', 'rotation_matrix', 'groups', 'object_bool']


In [4]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from model import *
from data_cleaning import all_groups, data_array

# Function to prepare fast arrays for Numba
def fast_arr(objects: np.ndarray):
    return np.array(
        [[object[0], object[4], object[6], object[13], 0, 0, 0] for object in objects]
    )

# Simulation function
def run_sim(
    objects: np.ndarray,
    group: int,
    draw: bool,
    margin: float,
    endtime: float,
    timestep: float,
    epoch: float,
    probability: float,
    percentage: float,
    frequency_new_debris: int,
    debris_start_time: float,
) -> tuple[list, list, list]:
    objects = objects[:99]
    initialize_positions(objects, epoch)
    objects_fast = fast_arr(objects)
    matrices = np.array([object[11] for object in objects])

    parameters, collisions, collision_times = [], [], []

    for time in tqdm(
        range(int(epoch), int(epoch + endtime), timestep),
        ncols=100,
        desc=f"group: {group}",
    ):
        calc_all_positions(objects_fast, matrices, time)
        collision_pairs = check_collisions_optimized(objects_fast, margin)
        if collision_pairs:
            for collided_objects in collision_pairs:
                _, _, object1, object2 = collided_objects
                collisions.append([object1, object2, time])
                collision_times.append(time)

        if (
            time >= debris_start_time
            and frequency_new_debris is not None
            and (time - epoch) % (frequency_new_debris * timestep) == 0
        ):
            objects_fast, matrices, _ = random_debris(
                objects_fast, matrices, time, percentage
            )

    parameters.append(
        [objects[0][12], epoch, endtime, timestep, probability, percentage]
    )
    return parameters, collisions, collision_times

# Function to plot launched satellites vs collisions
def plot_satellites_vs_collisions(launched, collisions):
    plt.figure(figsize=(10, 6))
    plt.plot(launched, collisions, marker="o", linestyle="-")
    plt.xlabel("Number of Satellites Launched")
    plt.ylabel("Total Collisions")
    plt.title("Number of Satellites Launched vs Total Collisions")
    plt.grid(True)
    plt.savefig("satellites_vs_collisions.png")
    plt.show()

if __name__ == "__main__":
    dataset_name = "Satellites"
    debris_start_time = 1675269600.0

    group = all_groups[0]  # Select the first group for simplicity
    group_selection = data_array[:, 12] == group
    data_array_group = data_array[group_selection]
    objects = data_array_group

    draw = False

    num_satellites_launched = []
    total_collisions = []

    for satellites_to_launch in range(0, 500, 50): 
        test_objects = objects[:satellites_to_launch]
        _, collisions, _ = run_sim(
            objects=test_objects,
            group=group,
            draw=draw,
            margin=5000,
            endtime=100_000,
            timestep=5,
            epoch=1675209600.0,
            probability=0,
            percentage=0,
            frequency_new_debris=40,
            debris_start_time=debris_start_time,
        )
        num_satellites_launched.append(satellites_to_launch)
        total_collisions.append(len(collisions))

    # Plot results
    plot_satellites_vs_collisions(num_satellites_launched, total_collisions)


group: 1:   0%|                                                           | 0/20000 [00:00<?, ?it/s]


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(float64, Literal[int](0))
 
There are 22 candidate implementations:
    - Of which 22 did not match due to:
    Overload of function 'getitem': File: <numerous>: Line N/A.
      With argument(s): '(float64, int64)':
     No match.

During: typing of intrinsic-call at /Users/zoeazra/Documents/CLS/Y1/Complex System Simulations/Github/ComplexSystemsProject/sim/model.py (175)
During: typing of static-get-item at /Users/zoeazra/Documents/CLS/Y1/Complex System Simulations/Github/ComplexSystemsProject/sim/model.py (175)

File "model.py", line 175:
def calc_all_positions(
    <source elided>
            time,
            epoch=objects[i][0],
            ^

During: Pass nopython_type_inference

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
import main
import data_cleaning

# Reload the modules to ensure the latest changes are picked up
reload(main)
reload(data_cleaning)

# Get data and groups from data_cleaning
data_array = data_cleaning.data_array
all_groups = data_cleaning.all_groups

# Select a group to run the simulation
group = all_groups[0]  # Replace with the desired group number if needed
group_selection = data_array[:, 12] == group
data_array_group = data_array[group_selection]

# Define simulation parameters (match those in main.py)
margin = 5000
endtime = 100_000
timestep = 5
epoch = 1675209600.0
probability = 0
percentage = 0
frequency_new_debris = 40

# Initialize data for plotting
num_satellites_launched = []
total_collisions = []

# Run the simulation with satellite launches
for satellites_to_launch in range(0, len(data_array_group), 50):  # Step by 50
    # Select a subset of satellites to simulate
    objects = data_array_group[:satellites_to_launch]

    # Run the simulation using main.run_sim
    _, collisions, _ = main.run_sim(
        objects=objects,
        group=group,
        draw=False,
        margin=margin,
        endtime=endtime,
        timestep=timestep,
        epoch=epoch,
        probability=probability,
        percentage=percentage,
        frequency_new_debris=frequency_new_debris,
    )

    # Log the results for plotting
    num_satellites_launched.append(len(objects))
    total_collisions.append(len(collisions))

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(num_satellites_launched, total_collisions, marker="o", linestyle="-")
plt.xlabel("Number of Satellites Launched")
plt.ylabel("Total Collisions")
plt.title("Number of Satellites Launched vs Total Collisions")
plt.grid(True)
plt.savefig("satellites_vs_collisions.png")
plt.show()

print("Plot saved as 'satellites_vs_collisions.png'.")


DATA CLEAN fully OBJECT Columns: ['EPOCH', 'INCLINATION', 'RA_OF_ASC_NODE', 'ARG_OF_PERICENTER', 'MEAN_ANOMALY', 'NORAD_CAT_ID', 'SEMIMAJOR_AXIS', 'OBJECT_TYPE', 'RCS_SIZE', 'LAUNCH_DATE', 'tuples', 'rotation_matrix', 'groups', 'object_bool']


group: 1:   0%|                                                           | 0/20000 [00:00<?, ?it/s]


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(float64, Literal[int](0))
 
There are 22 candidate implementations:
    - Of which 22 did not match due to:
    Overload of function 'getitem': File: <numerous>: Line N/A.
      With argument(s): '(float64, int64)':
     No match.

During: typing of intrinsic-call at /Users/zoeazra/Documents/CLS/Y1/Complex System Simulations/Github/ComplexSystemsProject/sim/model.py (175)
During: typing of static-get-item at /Users/zoeazra/Documents/CLS/Y1/Complex System Simulations/Github/ComplexSystemsProject/sim/model.py (175)

File "model.py", line 175:
def calc_all_positions(
    <source elided>
            time,
            epoch=objects[i][0],
            ^

During: Pass nopython_type_inference